# PBL02_OpenCV_testコード

PBL02_不良個所自動検出理解のためのOpenCVのサンプルコード

### 1. ライブラリのimport

In [ ]:
import cv2
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

### 2. 画像の取り込み

In [ ]:
img = cv2.imread("000_resize.jpeg") #ファイルを挿入

#画像はnpのarray形式で読み込まれる
print(img)

#OepnCVはBGR形式で読み込むがmatplotlibはRGB方式で取り込む必要があるため,BGR形式をRGB形式にconvertして表示させる
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

### 3. グレースケールへの変換

In [ ]:
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
print(img_gray)
plt.imshow(img_gray)

### 4. 閾値処理

閾値処理を使って基盤部分を黒く,背景部分を白く切り分ける

ピクセルの値がある閾値を超えた場合は,画像の各ピクセルを白いピクセルに,超えなかった場合は黒いピクセルに置き換える

In [ ]:
threshold = 110 #数値は試行錯誤で基板と背景が分かれるように適当に置いた
retval,img_threshold = cv2.threshold(img_gray,threshold,255,cv2.THRESH_BINARY)
plt.imshow(img_threshold)

- cv2.thresholdの戻り値について
    - retvalとthresholdの2つ
        - retvalは閾値の自動検出のための引数.はじめのうちは気にしなくてよさそう
        - thresholdは閾値処理されたあとの白黒の二値画像

### 5. ガウスぼかしによるノイズ除去

In [ ]:
#元画像
plt.imshow(img)

#5*5サイズのgaussianでのぼかし
img_gauss_5 = cv2.GaussianBlur(img,(5,5),0)
plt.imshow(img_gauss_5)

#15*15サイズのgaussianでのぼかし
img_gauss_15 = cv2.GaussianBlur(img,(15,15),0)
plt.imshow(img_gauss_15)

### 6. 輪郭の検出

グレースケールに変換して,ガウスぼかしで単純化とノイズ除去
閾値処理で基板部分と背景を白黒二値画像に変換

In [ ]:
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img_gauss = cv2.GaussianBlur(img_gray,(5,5),0)

threshold = 110
retval,img_threshold = cv2.threshold(img_gauss,threshold,255,cv2.THRESH_BINARY)

img_binary = cv2.bitwise_not(img_threshold)
plt.imshow(img_binary)

二値画像を用いた輪郭の検出

In [ ]:
contours, hierarchy = cv2.findContours(img_binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

- findContoursの引数について
    - RETR_EXTERNAL:輪郭のうち一番外側だけを検出する
    - CHAIN_APPROX_SIMPLE:輪郭の冗長な点を削除して必要最小限の点を検出する

- findContours の戻り値について
    - 今回はcontours, hierarchyの2つ
        - contours	オブジェクトの輪郭座標を保持している配列
        - hierarchy	オブジェクトの階層構造情報を保持している配列

検出するサイズの設定
- 今回は基板のみが検出できれば良いので検出された領域の大きさのうち,小さいものは取り除く

In [ ]:
min_area = 5000
large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
print(large_contours)
#輪郭データと画像データを重ねて,輪郭部を赤(255,0,0)で色付けする
img_contours = np.copy(img)
cv2.drawContours(img_contours, large_contours, -1, (255,0,0))

# 輪郭で検出された領域が1つしかないことを確認
# ここの戻り値が1でなければmin_areaの大きさを変更する
print('number of area: %d' % len(large_contours))
plt.imshow(img_contours)

今回の輪郭の検出では基板の影部分が認識されていたり,基板の厚さ部分が検知されていなかったりする.これは今後の課題

### 7. 外接矩形の検出
輪郭の検出で得られた輪郭データに外接する矩形を作成.これで基板だけを切り抜く前処理ができるはず

In [ ]:
img_bounding = np.copy(img)

x, y, w, h = cv2.boundingRect(large_contours[0])
cv2.rectangle(img_bounding, (x, y), (x + w, y + h), (255, 0, 0), 1)

plt.imshow(img_bounding)